In [361]:
import sys
sys.path

['',
 '/usr/lib/python35.zip',
 '/usr/lib/python3.5',
 '/usr/lib/python3.5/plat-x86_64-linux-gnu',
 '/usr/lib/python3.5/lib-dynload',
 '/home/tensae/.local/lib/python3.5/site-packages',
 '/usr/local/lib/python3.5/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.5/dist-packages/IPython/extensions',
 '/home/tensae/.ipython']

# Making some Atoms in python

Here we will mostly use the ipython shell to give examples. But you can of course put all the commands in a *script_name.py* file if you like and execute it, on terminal, using the command :

*$ python script_name.py* 

In [362]:
# Import AtomSpace and types to start making atoms.
from opencog.atomspace import AtomSpace, types

There are two ways to create Atoms in python. You can use the c++ like syntax or you can use the scheme like syntax.

# c++ like syntax

In [363]:
#declare an atomspace called atsp
atsp = AtomSpace()

In [364]:
#initializing atoms(nodes) of type ConceptNode.
cat = atsp.add_node(types.ConceptNode, "Cat")
animal = atsp.add_node(types.ConceptNode, "Animal")
man = atsp.add_node(types.ConceptNode, "Man")

In [365]:
#print a node
man

(ConceptNode "Man") ; [4943495822049051872][48]

Printing the AtomSpace will not quite work out as we might expect from c++. We can print its contents as follows:

In [366]:
#print the atomsapce
for atom in atsp:
    print (atom)

(ConceptNode "Animal")

(ConceptNode "Man")

(ConceptNode "Cat")



Now lets add some links into the AtomSpace. This is again, just like c++.

In [367]:
#initialize atom of type InheritanceLink.
atsp.add_link(types.InheritanceLink, [man, animal])

(InheritanceLink
  (ConceptNode "Man") ; [4943495822049051872][48]
  (ConceptNode "Animal") ; [1051245045201563895][48]
) ; [17225707082406533815][48]

In [368]:
def cplusplus_like_syntax ():
    """
    >>> man
    (ConceptNode "Man") ; [...][...]
    <BLANKLINE>
  
    >>> atsp.add_link(types.InheritanceLink, [man, animal])
    (InheritanceLink
      (ConceptNode "Man") ; [...][...]
      (ConceptNode "Animal") ; [...][...]
    ) ; [...][...]
    <BLANKLINE>
    """
    pass 

# scheme like syntax

To use scheme like concise syntax you have to do a few extra steps. Import the *type_constructors* that enable you to add atoms to the AtomSpace by calling functions named after their types. Import *initialize_opencog*, a function that binds an AtomSpace instance (*atsp* in our case) to the *type_constructors*.

In [369]:
#Import initialize_opencog
from opencog.utilities import initialize_opencog

In [370]:
#Import type_constructors
from opencog.type_constructors import *

Now you can add atoms just like in scheme. This becomes very useful when you start building complex patterns for the *pattern matcher*.

In [371]:
#bind atsp with the type_constructors
initialize_opencog(atsp)

In [372]:
#initialize and add more atoms into atsp
color = ConceptNode("Color")
c1 = ConceptNode("Blue")
link = InheritanceLink(c1, color)

In [373]:
#print the atomspace
for atom in atsp:
    print (atom)

(ConceptNode "Blue")

(ConceptNode "Cat")

(ConceptNode "Man")

(ConceptNode "Animal")

(ConceptNode "Color")

(InheritanceLink
  (ConceptNode "Blue")
  (ConceptNode "Color")
)

(InheritanceLink
  (ConceptNode "Man")
  (ConceptNode "Animal")
)



One last thing before we wrap this up. The Atoms are classes with a lot of useful methods. If you are in the ipython shell you can look at a list of the methods by writing *< variable_pointing_to_an_atom >* . *< Tab >*. For example, we have a variable called *"color"* that points to the *ConceptNode* object of name *"Color"*. Then we can see a list of all the methods of *"color"* by :

*color.< Press Tab >*

In [374]:
def test_AIO():
    """ test scheme like syntax, c++ like syntax and patternmatcher all in one.
    >>> human = atsp.add_node(types.ConceptNode, "human being") 
    >>> human # test for the c++ like syntax
    (ConceptNode "human being") ; [...][...]
    <BLANKLINE>
    >>> InheritanceLink(ConceptNode("Minilik"), human) # test for the scheme like syntax
    (InheritanceLink
      (ConceptNode "Minilik") ; [...][...]
      (ConceptNode "human being") ; [...][...]
    ) ; ...
    <BLANKLINE>
    >>> InheritanceLink(atsp.add_node(types.ConceptNode, "Tayitu"), human) # test for the combination of the two syntaxs
    (InheritanceLink
      (ConceptNode "Tayitu") ; [...][...]
      (ConceptNode "human being") ; [...][...]
    ) ; [...][...]
    <BLANKLINE>
    >>> variable_link = TypedVariableLink(VariableNode("$xpeople"), TypeNode("ConceptNode"))
    >>> patt = InheritanceLink(VariableNode("$xpeople"), human)
    >>> humans = GetLink(variable_link, patt)    
    >>> satisfying_set(atsp, humans) # test for the pattern matcher
    (SetLink
      (ConceptNode "Minilik") ; [...][...]
      (ConceptNode "Tayitu") ; [...][...]
    ) ; [...][...]
    <BLANKLINE>
    """
    pass

# Pattern Matching

Now lets look at how we can run the PM through python. The PM functions are defined in c++ and made available in python through wrappers that are defined in atomspace/opencog/cython/opencog/bindlink.pyx and you can see the imports in *atomspace/opencog/cython/opencog/bindlink.pxd*

Here we will again run the equivalent of *bindlink* and *cog-staisfying-set* functions in scheme.

In [375]:
#Import the satisfying_set function
from opencog.bindlink import satisfying_set

In [376]:
#Delete existing atoms
atsp.clear()
#NOTE that the atoms are deleted only from atsp not from the entire python environment. 

In [377]:
#Create some atoms
InheritanceLink(ConceptNode("Red"), color)
InheritanceLink(ConceptNode("Green"), color)
InheritanceLink(ConceptNode("Blue"), color)

(InheritanceLink
  (ConceptNode "Blue") ; [3109877522812845760][48]
  (ConceptNode "Color") ; [8429248309562626666][48]
) ; [10211166656246673930][48]

In [378]:
#Create a pattern to look for color nodes
varlink = TypedVariableLink(VariableNode("$xcol"), TypeNode("ConceptNode"))
pattern = InheritanceLink(VariableNode("$xcol"), color)
colornodes = GetLink(varlink, pattern)

In [379]:
#Now run the function to find all color nodes
satisfying_set(atsp, colornodes)

(SetLink
  (ConceptNode "Blue") ; [3109877522812845760][48]
  (ConceptNode "Red") ; [6709592507294448610][48]
  (ConceptNode "Green") ; [7362812596333203375][48]
) ; [17576040149376550613][48]

In a similar manner you can import *bindlink* from *opencog.bindlink* and use it to execute rewrite queries (with the help of *BindLink* in place of *SatisfactionLink*)

In [380]:
import doctest

In [381]:
doctest.testmod(verbose=True, optionflags=doctest.ELLIPSIS)

Trying:
    man
Expecting:
    (ConceptNode "Man") ; [...][...]
    <BLANKLINE>
ok
Trying:
    atsp.add_link(types.InheritanceLink, [man, animal])
Expecting:
    (InheritanceLink
      (ConceptNode "Man") ; [...][...]
      (ConceptNode "Animal") ; [...][...]
    ) ; [...][...]
    <BLANKLINE>
ok
Trying:
    man
Expecting:
    (ConceptNode "Man") ; [...][...]
    <BLANKLINE>
ok
Trying:
    human = atsp.add_node(types.ConceptNode, "human being") 
Expecting nothing
ok
Trying:
    human # test for the c++ like syntax
Expecting:
    (ConceptNode "human being") ; [...][...]
    <BLANKLINE>
ok
Trying:
    InheritanceLink(ConceptNode("Minilik"), human) # test for the scheme like syntax
Expecting:
    (InheritanceLink
      (ConceptNode "Minilik") ; [...][...]
      (ConceptNode "human being") ; [...][...]
    ) ; ...
    <BLANKLINE>
ok
Trying:
    InheritanceLink(atsp.add_node(types.ConceptNode, "Tayitu"), human) # test for the combination of the two syntaxs
Expecting:
    (InheritanceLink
  

TestResults(failed=0, attempted=11)